In [46]:
import requests
import urllib
from urllib import quote_plus, quote
from urlparse import urljoin
from bs4 import BeautifulSoup
import re

import pdb 

song_title="Bad Touch"
artist = "Bloodhound Gang"

if song_title is None:
    song_title = raw_input("Please enter a song title: ")
    artist = raw_input("Who is the artist?")

#if artist is not None:

originalTitle = song_title
originalArtist = artist

artist = re.sub("[\.&]","",artist)
artist = re.sub(" +","-",artist)
artist = re.sub("Featuring","feat",artist)
song_title = re.sub("\'","-",song_title)
song_title = re.sub(" ","-",song_title)

url2 = "https://www.musixmatch.com/lyrics/" + artist + \
                   "/" + song_title

print url2


response = urllib.urlopen(url2).read()
soup = BeautifulSoup(response,'lxml')

scripts = soup.find_all("script")

for script in scripts:
    tmp = script.text.split("__mxmState = ")
    if len(tmp)>1:
        data = tmp[1][:-1]
        jsdata = json.loads(data)
        break

#lyrics = re.findall("body\":\"(.*?)\"",response)

lyrics = jsdata['page']['lyrics']['lyrics']['body']

title = originalArtist + ": " + originalTitle
print "found lyrics on musixmatch!"
print lyrics[0]
#return lyrics, title




https://www.musixmatch.com/lyrics/Bloodhound-Gang/Bad-Touch
found lyrics on musixmatch!
Me and you do the kind of stuff that only Prince would sing about\nSo put your hands down my pants and I'll bet you'll feel nuts\n\nYes I'm Siskel yes I'm Ebert and you're getting two thumbs up\n\nYou've had enough of two-hand touch you want it rough you're out of bounds\nI want you smothered want you covered like my Waffle House hashbrowns\nComin' quicker than FedEx never reach an apex\njust like Coca-Cola stock you are inclined\nTo make me rise an hour early just like Daylight Savings Time\nDo it now\nYou and me baby ain't nothin' but mammals\nSo let's do it like they do on the Discovery Channel\nDo it again now\n\nYou and me baby ain't nothin' but mammals\nSo let's do it like they do on the Discovery Channel\nGettin' horny now\nLove the kind you clean up with a mop and bucket\nLike the lost catacombs of Egypt only God knows where we stuck it\nHieroglyphics? Let me be Pacific I wanna be down in yo

In [53]:
import json

#pattern = re.compile(ur'var __mxmState = (.*?);',re.UNICODE)

jsdata.keys()
# foo = tmp.text.split("__mxmState = ")[1][:-1]
# json.loads(foo)
# #print tmp.prettify



[u'embedly', u'lyrics', u'page']

In [62]:


lyrics = jsdata['page']['lyrics']['lyrics']['body']



In [65]:
print lyrics

I'd appreciate your input.
Sweat baby sweat baby sex is a Texas drought
Me and you do the kind of stuff that only Prince would sing about

So put your hands down my pants and I'll bet you'll feel nuts

Yes I'm Siskel yes I'm Ebert and you're getting two thumbs up
You've had enough of two-hand touch you want it rough you're out of bounds
I want you smothered want you covered like my Waffle House hashbrowns
Comin' quicker than FedEx never reach an apex
just like Coca-Cola stock you are inclined
To make me rise an hour early just like Daylight Savings Time
Do it now
You and me baby ain't nothin' but mammals
So let's do it like they do on the Discovery Channel

Do it again now
You and me baby ain't nothin' but mammals
So let's do it like they do on the Discovery Channel
Gettin' horny now
Love the kind you clean up with a mop and bucket
Like the lost catacombs of Egypt only God knows where we stuck it
Hieroglyphics? Let me be Pacific I wanna be down in your South Seas

But I got this notion

In [1]:
import requests
import urllib
from urllib import quote_plus, quote
from urlparse import urljoin
from bs4 import BeautifulSoup
import re
import json

import pdb 

def get_song_lyrics(song_title=None,artist=None):

    #pdb.set_trace()
    if song_title is None:
        song_title = raw_input("Please enter a song title: ")
        artist = raw_input("Who is the artist?")
        
    if artist is not None:

        originalTitle = song_title
        originalArtist = artist

        artist = re.sub("[\.&]","",artist)
        artist = re.sub(" +","-",artist)
        artist = re.sub("Featuring","feat",artist)
        song_title = re.sub("\'","-",song_title)
        song_title = re.sub(" ","-",song_title)

        url2 = "https://www.musixmatch.com/lyrics/" + artist + \
                           "/" + song_title

        print url2

        try:
            response = urllib.urlopen(url2).read()
            soup = BeautifulSoup(response,'lxml')
            tmp = soup.find_all("script")

            lyrics = re.findall("body\":\"(.*?)\"",response)

            title = originalArtist + ": " + originalTitle
            print "found lyrics on musixmatch!"
            return lyrics, title
        except:
            print "couldn't find on musixmatch"
            pass

    print "searching chartlyrics"
    BASE_URL = "http://www.chartlyrics.com/"
    #queryString = "track.search?q_track=" #chartlyrics would return a soap object if i properly queried it
    searchURL = BASE_URL + "search.aspx?q=" + song_title
    
    fullurl = quote_plus(searchURL, safe="%/:=&?~#!+$,;'@()*[]")
    #fullurl = quote(searchURL, safe="%/:=&?~#!+$,;'@()*[]")

    response = requests.get(fullurl)
    soup = BeautifulSoup(response.text,'lxml')

    tmp = soup.find_all('a')
    song_link = tmp[5];

    # i could also do soup.find('table')

    link = urljoin(BASE_URL, song_link['href'])
    response = requests.get(link);

    soup = BeautifulSoup(response.text,'lxml')
    lyrics_box = soup.find('p')#.text.strip()

    try:
        lyrics = lyrics_box.text.strip()

        title = soup.find('head').find('title').text

        # re.findall("u\'(.*)\\t",

    except:
        
            print "Couldn't find the lyrics. Try another song."
            return
    return lyrics, title




In [7]:
'''some interesting charts- heatseekers-songs, r-b-hip-hop-songs, rock-songs, dance-electronic-songs, 
country-songs, adult-contemporary

We are making a csv out of the jsons'''

import billboard
from time import sleep
from dateutil.parser import parse as parse_date
import json
import csv

import pdb

def build_chart_db(chartName='hot-100',filename = 'billboard_data.csv', 
                   StartDate='6-1-2016',EndDate=None,date=None,append=True):
    
    if append:
        f = csv.writer(open(filename,"awb+"))
    else:
        f = csv.writer(open(filename,"wb+"))
        
    if date is None and StartDate is None and EndDate is None:
        chart = billboard.ChartData(chartName)
    # we will do some elifs later    
    
    elif StartDate:
        
        chart = billboard.ChartData(chartName)
        
        if EndDate:    
            while chart.previousDate>EndDate:
                chart = billboard.ChartData(chartName,previousDate)
        
       
        #pdb.set_trace()
        while parse_date(chart.date)>parse_date(StartDate):
            js1 = chart.to_JSON()
            bigDict = json.loads(js1)

            f.writerow(['chart_name','date','title','artist','rank','weeks','peakPos','lastPos','change','spotifyLink'])

            for entry in bigDict['entries']:
                
                f.writerow([bigDict['name'],bigDict['date'],entry['title'],entry['artist'],entry['rank'],entry['weeks'],
                      entry['peakPos'],entry['lastPos'],entry['change'],entry['spotifyLink']])
            
            chart = billboard.ChartData(chartName,chart.previousDate)
            sleep(1)
            
    

In [15]:
chart = billboard.ChartData('hot-100')

chart.date


u'2016-06-18'

In [9]:
build_chart_db()
data = open('billboard_data.csv')
dataRead = csv.reader(data)
list(dataRead)

[['chart_name',
  'date',
  'title',
  'artist',
  'rank',
  'weeks',
  'peakPos',
  'lastPos',
  'change',
  'spotifyLink'],
 ['hot-100',
  '2016-06-18',
  'One Dance',
  'Drake Featuring WizKid & Kyla',
  '1',
  '9',
  '1',
  '1',
  '0',
  'https://embed.spotify.com/?uri=spotify:track:11hqMWwX7sF3sOGdtijofF'],
 ['hot-100',
  '2016-06-18',
  'Panda',
  'Desiigner',
  '2',
  '15',
  '1',
  '2',
  '0',
  'https://embed.spotify.com/?uri=spotify:track:6r2jK1A6oFRPREZfxjc5d1'],
 ['hot-100',
  '2016-06-18',
  "Can't Stop The Feeling!",
  'Justin Timberlake',
  '3',
  '4',
  '1',
  '3',
  '0',
  'https://embed.spotify.com/?uri=spotify:track:6JV2JOEocMgcZxYSZelKcc'],
 ['hot-100',
  '2016-06-18',
  'Work From Home',
  'Fifth Harmony Featuring Ty Dolla $ign',
  '4',
  '14',
  '4',
  '4',
  '0',
  'https://embed.spotify.com/?uri=spotify:track:0SJPThTy7ynySPF4euczx7'],
 ['hot-100',
  '2016-06-18',
  "Don't Let Me Down",
  'The Chainsmokers Featuring Daya',
  '5',
  '16',
  '5',
  '5',
  '0',
  'h

In [ ]:
def build_chart_db(chartName='hot-100',filename = 'billboard_data.csv', 
                   StartDate='6-1-2016',EndDate=None,date=None,append=False):
    
    if append:
        fr = open(filename,"r")
        f_rw = (open(filename,"awb+"))
        f = csv.writer(f_rw)
        fr = csv.reader(fr)
    else:
        if os.path.isfile(filename):
            r = raw_input("This file already exists and append=False. Do you want to overwrite it? \(y/n\) :")
            if r.lower='y':
                f = csv.writer(open(filename,"wb+"))
            else:
                return
        
    if date is None and StartDate is None and EndDate is None:
        chart = billboard.ChartData(chartName)
    # we will do some elifs later    
    elif StartDate:
        
        chart = billboard.ChartData(chartName)

        if not append:
            f.writerow(['chart_name','date','title','artist','rank','weeks','peakPos','lastPos','change','spotifyLink'])

        #pdb.set_trace()
        while parse_date(chart.date)>parse_date(StartDate):
            js1 = chart.to_JSON()
            bigDict = json.loads(js1)

            #f.writerow(['chart_name','date','title','artist','rank','weeks','peakPos','lastPos','change','spotifyLink'])

            for entry in bigDict['entries']:
                if append:
                    # check if this chart has already been added
                    
                    
                f.writerow([bigDict['name'],bigDict['date'],entry['title'],entry['artist'],entry['rank'],entry['weeks'],
                      entry['peakPos'],entry['lastPos'],entry['change'],entry['spotifyLink']])
            
            chart = billboard.ChartData(chartName,chart.previousDate)
            sleep(2)

In [16]:
import billboard
from dateutil.parser import parse as parse_date
import json
import csv

chart = billboard.ChartData('dance-electronic-songs')

#parse_date(chart.date) > parse_date('1-1-2016')

js1 = chart.to_JSON()
bigDict = json.loads(js1)

fname = 'billboard_data.csv'

f = csv.writer(open(fname,"awb+"))

f.writerow(['chart_name','date','title','artist','rank','weeks','peakPos','lastPos','change','spotifyLink'])

for entry in bigDict['entries']:
    f.writerow([bigDict['name'],bigDict['date'],entry['title'],entry['artist'],entry['rank'],entry['weeks'],
          entry['peakPos'],entry['lastPos'],entry['change'],entry['spotifyLink']])
    

# chart2 = billboard.ChartData('dance-electronic-songs',chart.previousDate)
# js2 = chart2.to_JSON()
# newDict = json.loads(js2)
# #bigDict.update(newDict)
# jstmp= js1 + '\n' + js2

#print js1
#json.dump?
#print json.dump(bigDict) 


In [23]:
'''this guy is going to take title and artist from the chart db and make lyric features for every song; 
this will then be joined to the other guy in a big data frame maybe? or csv
'''
#print js1
with open("billboard_data.csv") as f:
    tmp = csv.reader(f)
    
    list(tmp)
    #def build_lyric_db_from_chart():

In [6]:


import billboard

chart = billboard.ChartData('hot-100')

In [25]:
import billboard
# this does not have the position info so billboard.py craps out
chart = billboard.ChartData('year-end/2015/hot-100-songs') 

AttributeError: 'NoneType' object has no attribute 'string'

In [26]:
import pandas as pd

def build_lyrics_db_from_chart(filename="billboard_data.csv",fnameTarget = "lyrics.csv"):
    dfIn = pd.from_csv(filename)
    dfOut = dfIn.
    

In [96]:
import pandas as pd
import numpy as np
filename="billboard_data.csv"
dfIn = pd.read_csv(filename)
dfIn.drop_duplicates(subset = ["title","artist"])

#dfOut = dfIn[['title','artist']]
#dfOut.columns
#dfOut = dfIn['lyrics'].apply(lambda row: get_song_lyrics(row['title'],row['artist'], axis=1))
#arra
lyricsDF = pd.DataFrame(columns=["lyrics","url"], index = dfIn.index)
#title
# lyricsDF.loc[0]['lyrics'] = lyrics
# lyricsDF.loc[0]['url'] = url
# lyricsDF.loc[0]
# #dfIn.columns

for index, title, artist in zip(dfIn.index, dfIn['title'],dfIn['artist']):
    print(index,title,artist)
    
    try:
        lyrics, url = get_MXMsong_lyrics(title,artist)
        lyricsDF.loc[index]['lyrics'] = lyrics
        lyricsDF.loc[index]['url'] = url
    except:
        lyricsDF.loc[index]['lyrics'] = np.nan
        lyricsDF.loc[index]['url'] = np.nan
        continue
    
# for row in dfIn.iterrows():
    
#     title = row[1][1]
#     artist = row[1][2]
#     print(title, artist)
    
#     lyrics, title, url = get_song_lyrics(row[0]['title'],row[0]['artist'])
#     lyricsDF.loc[index]['lyrics'] = lyrics
#     lyricsDF.loc[index]['url'] = url
    
    

(0, 'One Dance', 'Drake Featuring WizKid & Kyla')
https://www.musixmatch.com/lyrics/Drake-feat-WizKid-Kyla/One-Dance
(1, 'Panda', 'Desiigner')
https://www.musixmatch.com/lyrics/Desiigner/Panda
(2, "Can't Stop The Feeling!", 'Justin Timberlake')
https://www.musixmatch.com/lyrics/Justin-Timberlake/Can-t-Stop-The-Feeling
(3, 'Work From Home', 'Fifth Harmony Featuring Ty Dolla $ign')
https://www.musixmatch.com/lyrics/Fifth-Harmony-feat-Ty-Dolla-ign/Work-From-Home
(4, "Don't Let Me Down", 'The Chainsmokers Featuring Daya')
https://www.musixmatch.com/lyrics/The-Chainsmokers-feat-Daya/Don-t-Let-Me-Down
(5, '7 Years', 'Lukas Graham')
https://www.musixmatch.com/lyrics/Lukas-Graham/7-Years
(6, 'I Took A Pill In Ibiza', 'Mike Posner')
https://www.musixmatch.com/lyrics/Mike-Posner/I-Took-A-Pill-In-Ibiza
(7, 'Needed Me', 'Rihanna')
https://www.musixmatch.com/lyrics/Rihanna/Needed-Me
(8, 'This Is What You Came For', 'Calvin Harris Featuring Rihanna')
https://www.musixmatch.com/lyrics/Calvin-Harris-f

In [112]:

#print lyricsDF.loc[0]['lyrics']
billboard_and_lyrics_df = pd.concat([dfIn, lyricsDF],1)

billboard_and_lyrics_df.to_csv('lyrics_and_billboard.csv', sep=',', encoding='utf-8')


In [95]:
import requests
import urllib
from urllib import quote_plus, quote
from urlparse import urljoin
from bs4 import BeautifulSoup
import re
import json

import pdb
    
def get_MXMsong_lyrics(song_title=None, artist=None):

    originalTitle = song_title
    originalArtist = artist

    if originalArtist is "P!nk":
        artist = "P-nk-2"
        
    artist = re.sub("[\.&$]","",artist)
    artist = re.sub(" +","-",artist)
    artist = re.sub("Featuring","feat",artist)
    song_title = re.sub("\'","-",song_title)
    song_title = re.sub(" ","-",song_title)
    song_title = re.sub("!","",song_title)

    url2 = "https://www.musixmatch.com/lyrics/" + artist + \
                       "/" + song_title

    print url2


    response = urllib.urlopen(url2).read()
    soup = BeautifulSoup(response,'html')
    #print soup.prettify()

    #type(soup)
    scripts = soup.findAll('script')
    for script in scripts:
        tmp = script.text.split("__mxmState = ")
        if len(tmp)>1:
            data = tmp[1][:-1]
            jsdata = json.loads(data)
            
    try:
        lyrics = jsdata['page']['lyrics']['lyrics']['body']
        return lyrics, url2
    except:
        print "couldn't find the lyrics on musixmatch"

In [18]:
#get_song_lyrics(dfIn['title'][0], dfIn['artist'][0])
title = dfIn['title'][0]
artist = dfIn['artist'][0]
title = "sandstorm"
artist = "darude"
get_song_lyrics(title,artist)

that's an instrumental track!


('Instrumental',
 'darude: sandstorm',
 'https://www.musixmatch.com/lyrics/darude/sandstorm')

In [80]:
dfIn[['artist','title']]

,artist,title
0,Drake Featuring WizKid & Kyla,One Dance
1,Desiigner,Panda
2,Justin Timberlake,Can't Stop The Feeling!
3,Fifth Harmony Featuring Ty Dolla $ign,Work From Home
4,The Chainsmokers Featuring Daya,Don't Let Me Down
5,Lukas Graham,7 Years
6,Mike Posner,I Took A Pill In Ibiza
7,Rihanna,Needed Me
8,Calvin Harris Featuring Rihanna,This Is What You Came For
9,P!nk,Just Like Fire
